In [ ]:
import pandas as pd
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/DrHeh/Google Drive/programming/py_DataIncubator/Medicare-a439a4c815d3.json"
import bq_helper
from bq_helper import BigQueryHelper
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
medicare = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="cms_medicare")
bq_assistant = BigQueryHelper("bigquery-public-data", "cms_medicare")
query_OP = """SELECT
  provider_id, 
  provider_state, 
  provider_city, 
  provider_zipcode, 
  average_total_payments, 
  outpatient_services, apc
  
FROM
  `bigquery-public-data.cms_medicare.outpatient_charges_2011`
  
UNION ALL (SELECT
  provider_id, 
  provider_state, 
  provider_city, 
  provider_zipcode, 
  average_total_payments, 
  outpatient_services, apc
  
FROM
  `bigquery-public-data.cms_medicare.outpatient_charges_2012`)
  
UNION ALL (SELECT
  provider_id, 
  provider_state, 
  provider_city, 
  provider_zipcode, 
  average_total_payments, 
  outpatient_services, apc
  
FROM
  `bigquery-public-data.cms_medicare.outpatient_charges_2013`)
  
UNION ALL (SELECT
  provider_id, 
  provider_state, 
  provider_city, 
  provider_zipcode, 
  average_total_payments, 
  outpatient_services, apc
  
FROM
  `bigquery-public-data.cms_medicare.outpatient_charges_2014`)
  
UNION ALL (SELECT
  provider_id, 
  provider_state, 
  provider_city, 
  provider_zipcode, 
  average_total_payments, 
  outpatient_services, apc
  
FROM
  `bigquery-public-data.cms_medicare.outpatient_charges_2015`)


ORDER BY
  provider_id DESC
;
        """
DFquery_OP = medicare.query_to_pandas_safe(query_OP)
DFquery_OP.head(50)
DFquery_OPBillProverService = DFquery_OP.pivot_table(index=["provider_id"], 
                                                     columns=["apc"], values=["average_total_payments"])

cmap = sns.cubehelix_palette(as_cmap=True, light=0.8)
sns.heatmap(DFquery_OPBillProverService, xticklabels=False, yticklabels=False,cmap=cmap)
plt.title('Out Patient Charges -- US dollar')
plt.savefig("Out Patient Charges.png", dpi=300)
plt.show()
df_norm_col=(DFquery_OPBillProverService-DFquery_OPBillProverService.mean())/DFquery_OPBillProverService.std()
sns.heatmap(df_norm_col, xticklabels=False, yticklabels=False, cmap='viridis')
plt.title('Out Patient Charges -- Normalized in each apc')
plt.savefig("Out Patient Charges_norm.png", dpi=300)
plt.show()
df_norm = df_norm_col[df_norm_col>0.01].dropna(how="all").fillna(0)
df_norm["ChargeScore"]=df_norm.sum(axis=1)
df_HiChargeProvider = df_norm[(df_norm.ChargeScore>=15)]
df_Hi = DFquery_OP[DFquery_OP.provider_id.isin(df_HiChargeProvider.index)]
df_Hi["Class"]="HiCharge"
df_Regular = DFquery_OP[~DFquery_OP.provider_id.isin(df_HiChargeProvider.index)]
df_Regular["Class"]="RegularCharge"
df = pd.concat([df_Hi, df_Regular], axis=0)
dfClassBy=df.pivot_table(index=["provider_state","Class","provider_id"],aggfunc="mean").round(1)
dfClassBy.reset_index(inplace=True)
dfbyStateProviders = dfClassBy[["provider_state","Class","provider_id"]].pivot_table(index=["provider_state","Class"],
                                                               values=["provider_id"],aggfunc=lambda x:len(x))
dfbyStateProviders.reset_index(inplace=True)
dfbyStateProviders = dfbyStateProviders.fillna(0)
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g= sns.catplot(data = dfbyStateProviders, kind="bar", height = 6, aspect = 2, 
               x="provider_state", y= "provider_id", hue="Class",edgecolor="0.1",palette="Set1")
plt.xlabel('State', fontsize=15)
plt.ylabel('Number of providers',fontsize=15)
g.set_xticklabels(rotation=45, fontsize=11)
g.savefig('ProvidersbyState.png', dpi=300)
plt.show()
dfbyapc=df.pivot_table(index=["apc","Class"],aggfunc= lambda x:len(x)).round(1)
dfbyapc.reset_index(inplace=True)
dfbyapc2 = dfbyapc[["apc","Class","average_total_payments"]].rename(columns={"average_total_payments":"counts"})
dfbyapc2 = dfbyapc2.replace({"Class":{"HiCharge":"From HiCharge Providers","RegularCharge":"From RegularCharge Providers"}})
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g= sns.catplot(data = dfbyapc2, kind="bar", height = 6, aspect = 2, 
               x="apc", y= "counts", hue="Class",edgecolor="0.1",palette="Set2")
plt.xlabel('apc', fontsize=15)
plt.ylabel('Number of files',fontsize=15)
g.set_xticklabels(rotation=90, fontsize=11)
g._legend.set_title("")
g.savefig('Providersbyapc.png', dpi=300)
plt.show()
query_quality = """SELECT
  provider_id, 
  hospital_ownership, 
  meets_criteria_for_meaningful_use_of_ehrs, 
  hospital_overall_rating, 
  mortality_national_comparison,
  safety_of_care_national_comparison, 
  readmission_national_comparison,
  patient_experience_national_comparison,
  effectiveness_of_care_national_comparison,
  timeliness_of_care_national_comparison,
  efficient_use_of_medical_imaging_national_comparison 
FROM
  `bigquery-public-data.cms_medicare.hospital_general_info`
  

ORDER BY
  provider_id DESC
;
        """
dfQuality = medicare.query_to_pandas_safe(query_quality)
dfQuality.head(50)
dfQuality_Hi = dfQuality[dfQuality.provider_id.isin(df_Hi.provider_id)]
dfQuality_Hi["Class"]="HiCharge"
dfQuality_Regular = dfQuality[dfQuality.provider_id.isin(df_Regular.provider_id)]
dfQuality_Regular["Class"]="RegularCharge"
dfQuality = pd.concat([dfQuality_Regular,dfQuality_Hi])
dfQuality = dfQuality.replace({"mortality_national_comparison":{"Not Avaiable":"na",
                                                                    "Above the national average":"Above Average",
                                                                    "Same as the national average":"Average",
                                                                    "Below the national average":"Below Average",}})
sns.set(style = "ticks", font_scale = 1.0, color_codes=True)
sns.set_context("talk")
g= sns.countplot(data = dfQuality,y="mortality_national_comparison", hue="Class",edgecolor=(0,0,0),
                 saturation=1, palette="muted")
g.legend_.remove()
plt.xlabel('Number of providers', fontsize=15)
plt.ylabel('Mortality',fontsize=15)
plt.xticks(rotation=45, horizontalalignment='right',fontsize=15)
plt.yticks(horizontalalignment='right',fontsize=15)
plt.tight_layout()
g.figure.savefig('mortality.png', dpi=100)
plt.show()
dfQuality = dfQuality.replace({"safety_of_care_national_comparison":{"Not Avaiable":"na",
                                                                    "Above the national average":"Above Average",
                                                                    "Same as the national average":"Average",
                                                                    "Below the national average":"Below Average",}})
sns.set(style = "ticks", font_scale = 1.0, color_codes=True)
sns.set_context("talk")
g= sns.countplot(data = dfQuality,y="safety_of_care_national_comparison", hue="Class",edgecolor=(0,0,0),
                 saturation=1, palette="muted")
g.legend_.remove()
plt.xlabel('Number of providers', fontsize=15)
plt.ylabel('Safety of Care',fontsize=15)
plt.xticks(rotation=45, horizontalalignment='right',fontsize=15)
plt.yticks(horizontalalignment='right',fontsize=15)
plt.tight_layout()
g.figure.savefig('SafetyCare.png', dpi=100)
plt.show()
dfQuality = dfQuality.replace({"readmission_national_comparison":{"Not Avaiable":"na",
                                                                    "Above the national average":"Above Average",
                                                                    "Same as the national average":"Average",
                                                                    "Below the national average":"Below Average",}})
sns.set(style = "ticks", font_scale = 1.0, color_codes=True)
sns.set_context("talk")
g= sns.countplot(data = dfQuality,y="readmission_national_comparison", hue="Class",edgecolor=(0,0,0),
                 saturation=1, palette="muted")
#g.legend(bbox_to_anchor=(0.75, 1.3), frameon=False, ncol=2)
g.legend_.remove()
plt.xlabel('Number of providers', fontsize=15)
plt.ylabel('Readmission',fontsize=15)
plt.xticks(rotation=45, horizontalalignment='right',fontsize=15)
plt.yticks(horizontalalignment='right',fontsize=15)
plt.tight_layout()
g.figure.savefig('Readmission.png', dpi=100)
plt.show()
dfQuality = dfQuality.replace({"patient_experience_national_comparison":{"Not Avaiable":"na",
                                                                    "Above the national average":"Above Average",
                                                                    "Same as the national average":"Average",
                                                                    "Below the national average":"Below Average",}})
sns.set(style = "ticks", font_scale = 1.0, color_codes=True)
sns.set_context("talk")
g= sns.countplot(data = dfQuality,y="effectiveness_of_care_national_comparison", hue="Class",edgecolor=(0,0,0),
                 saturation=1, palette="muted")
g.legend_.remove()
plt.xlabel('Number of providers', fontsize=15)
plt.ylabel('Effectiveness of Care',fontsize=15)
plt.xticks(rotation=45, horizontalalignment='right',fontsize=15)
plt.yticks(horizontalalignment='right',fontsize=15)
plt.tight_layout()
g.figure.savefig('EffectivenessofCare.png', dpi=100)
plt.show()
dfQuality = dfQuality.replace({"timeliness_of_care_national_comparison":{"Not Avaiable":"na",
                                                                    "Above the national average":"Above Average",
                                                                    "Same as the national average":"Average",
                                                                    "Below the national average":"Below Average",}})
sns.set(style = "ticks", font_scale = 1.0, color_codes=True)
sns.set_context("talk")
g= sns.countplot(data = dfQuality,y="timeliness_of_care_national_comparison", hue="Class",edgecolor=(0,0,0),
                 saturation=1, palette="muted")
g.legend_.remove()
plt.xlabel('Number of providers', fontsize=15)
plt.ylabel('Timeliness of Care',fontsize=15)
plt.xticks(rotation=45, horizontalalignment='right',fontsize=15)
plt.yticks(horizontalalignment='right',fontsize=15)
plt.tight_layout()
g.figure.savefig('Timeliness.png', dpi=100)
plt.show()
dfQuality = dfQuality.replace({"efficient_use_of_medical_imaging_national_comparison":{"Not Avaiable":"na",
                                                                    "Above the national average":"Above Average",
                                                                    "Same as the national average":"Average",
                                                                    "Below the national average":"Below Average",}})
sns.set(style = "ticks", font_scale = 1.0, color_codes=True)
sns.set_context("talk")
g= sns.countplot(data = dfQuality,y="efficient_use_of_medical_imaging_national_comparison", hue="Class",edgecolor=(0,0,0),
                 saturation=1, palette="muted")
g.legend_.remove()
plt.xlabel('Number of providers', fontsize=15)
plt.ylabel('Use Imaging Efficiently',fontsize=15)
plt.xticks(rotation=45, horizontalalignment='right',fontsize=15)
plt.yticks(horizontalalignment='right',fontsize=15)
plt.tight_layout()
g.figure.savefig('EfficientUseimaging.png', dpi=100)
plt.show()
sns.set(style = "ticks", font_scale = 1.0, color_codes=True)
sns.set_context("talk")
g= sns.countplot(data = dfQuality,y="hospital_overall_rating", hue="Class",edgecolor=(0,0,0),
                 saturation=1, palette="muted")
g.legend_.remove()
plt.xlabel('Number of providers', fontsize=15)
plt.ylabel('Hospital Overall Rating',fontsize=15)
plt.xticks(rotation=45, horizontalalignment='right',fontsize=15)
plt.yticks(horizontalalignment='right',fontsize=15)
plt.tight_layout()
g.figure.savefig('hospital_overall_rating.png', dpi=100)
plt.show()